In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from pandas_summary import DataFrameSummary
from pathlib import Path

import surprise
import os, math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 400)
sns.set()
os.chdir('/home/krivas/projects/analysis-project/')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
DATA        = Path('data')
RAW         = DATA/'raw'
PROCESSED   = DATA/'processed'

# Reading data

In [6]:
csv = pd.read_csv(PROCESSED/'HistoricoJob.csv')

# Testing

In [5]:
from surprise import Reader, Dataset
reader = Reader(rating_scale=(0, 1))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(csv[['Id_Job', 'Id_Malla', 'Termino']], reader)

In [12]:
data.split(n_folds=5)

from surprise import SVD, NMF, KNNBasic, evaluate

In [9]:
# svd
algo = SVD()
evaluate(algo, data, measures=['RMSE'])

/home/krivas/anaconda3/lib/python3.6/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/home/krivas/anaconda3/lib/python3.6/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE of algorithm SVD.

------------
Fold 1
RMSE: 0.0561
------------
Fold 2
RMSE: 0.0565
------------
Fold 3
RMSE: 0.0566
------------
Fold 4
RMSE: 0.0569
------------
Fold 5
RMSE: 0.0573
------------
------------
Mean RMSE: 0.0567
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.05610950655106352,
                             0.056513692931998455,
                             0.056571367873535956,
                             0.05686919071915562,
                             0.057274388159718664]})

In [10]:
# nmf
algo = NMF()
evaluate(algo, data, measures=['RMSE'])

/home/krivas/anaconda3/lib/python3.6/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/home/krivas/anaconda3/lib/python3.6/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE of algorithm NMF.

------------
Fold 1
RMSE: 0.0595
------------
Fold 2
RMSE: 0.0597
------------
Fold 3
RMSE: 0.0609
------------
Fold 4
RMSE: 0.0602
------------
Fold 5
RMSE: 0.0605
------------
------------
Mean RMSE: 0.0602
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.05950251521482442,
                             0.059745619520666406,
                             0.060911096013864297,
                             0.06016750579425097,
                             0.06053049532929203]})

In [18]:
dict_data = surprise.model_selection.split.train_test_split(data)

In [26]:
dict_data[0]

# NMF

In [36]:
from surprise import NMF

nmf = NMF(n_factors=15, n_epochs=50, biased=True, verbose=True)
reader = Reader(rating_scale=(0, 1))
data   = Dataset.load_from_df(csv[['Id_Job', 'Id_Malla', 'Termino']].sample(frac=0.1, random_state=30), reader)
trainset = data.build_full_trainset()